In [1]:
import pandas as pd 
import numpy as np 


In [3]:
data = pd.read_csv(r"./data/gem.csv/train.csv")

In [4]:
data.drop(labels=["id"],axis=1,inplace=True)

In [17]:
y = data["price"]
X=data.drop(labels=["price"],axis=1)

In [8]:
categorical_cols = X.select_dtypes(include='object').columns

In [9]:
numerical_cols=X.select_dtypes(exclude='object').columns

In [10]:
X.select_dtypes(include='object')

,cut,color,clarity
0,Premium,F,VS2
1,Very Good,J,SI2
2,Ideal,G,VS1
3,Ideal,G,VS1
4,Premium,G,VS2
...,...,...,...
193568,Ideal,D,VVS2
193569,Premium,G,VVS2
193570,Very Good,F,SI1
193571,Very Good,D,SI1


In [11]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [12]:
from sklearn.impute import SimpleImputer ## for handling missing values
from sklearn.preprocessing import StandardScaler ## handling feature scaling
from sklearn.preprocessing import OrdinalEncoder # ordinal encoding
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [13]:
## In pipeline we simply create the objects in steps i.e., initially as we will handle missing value, so we will write SimpleImputer, 
## in 2nd step  we will standardize the data, we will create standard scaler 
## We write under steps which is of type list
## Then we pass the categories in categorical_pipeline for encoding in cat_pipeline

num_pipeline = Pipeline(
    
    
    steps = [
        
        ('imputer',SimpleImputer()),
        ('scaler', StandardScaler())
        
    ]
    
)

In [14]:
cat_pipeline = Pipeline(
    
    steps = [
        
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinal encoder', OrdinalEncoder(categories = [cut_categories,color_categories,clarity_categories]))
    
    ]
    
    
)

In [16]:
## Now transform the data i.e., transformation is applied on pipeline
## Num_pipeline will be applied on numerical column and cat_pipeline will be applied on data in categorical columns
 
preprocessor = ColumnTransformer(
    [
       
        ('num_pipeline', num_pipeline,numerical_cols),
        ('cat_pipeline', cat_pipeline, categorical_cols)
        
    ]
)

In [18]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X, y , test_size=0.30,random_state=30)

In [21]:
## now transform is applied to preprocessor
preprocessor.fit_transform(X_train)

array([[-0.97543926, -0.84960654, -0.12153081, ...,  4.        ,
         5.        ,  5.        ],
       [ 0.2351953 ,  1.83363716, -0.12153081, ...,  1.        ,
         1.        ,  2.        ],
       [ 0.49461699,  0.81585507,  0.39980029, ...,  3.        ,
         3.        ,  4.        ],
       ...,
       [ 0.45138004,  1.55606023, -0.6428619 , ...,  1.        ,
         3.        ,  2.        ],
       [ 0.66756478, -1.77486298,  1.44246248, ...,  4.        ,
         3.        ,  4.        ],
       [ 0.25681377,  0.81585507, -0.12153081, ...,  4.        ,
         3.        ,  2.        ]])

In [22]:
preprocessor.get_feature_names_out()

array(['num_pipeline__carat', 'num_pipeline__depth',
       'num_pipeline__table', 'num_pipeline__x', 'num_pipeline__y',
       'num_pipeline__z', 'cat_pipeline__cut', 'cat_pipeline__color',
       'cat_pipeline__clarity'], dtype=object)

In [23]:
preprocessor.transform(X_test)


array([[-0.56468825, -0.94213218, -0.6428619 , ...,  3.        ,
         1.        ,  3.        ],
       [-0.17555571,  1.00090636, -0.12153081, ...,  2.        ,
         4.        ,  2.        ],
       [-1.06191316,  0.2607012 , -0.12153081, ...,  4.        ,
         4.        ,  7.        ],
       ...,
       [-0.19717419, -3.34779894,  1.44246248, ...,  3.        ,
         6.        ,  3.        ],
       [-0.82410994, -0.20192702, -0.12153081, ...,  4.        ,
         3.        ,  2.        ],
       [ 2.61322747, -0.75708089,  1.44246248, ...,  3.        ,
         6.        ,  3.        ]])

In [24]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [25]:
# linear, ridge, lasso, elastic regression 

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [26]:
## Train multiple models

models ={
    'LinearRegression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'ElasticNet' :ElasticNet()
}

trained_model_list = []
model_list = []
r2_list = []

In [28]:
list(models)

['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet']

In [27]:
for i in range(len(list(models))):
    print(i)

0
1
2
3


In [29]:
models.keys()

dict_keys(['LinearRegression', 'Lasso', 'Ridge', 'ElasticNet'])

In [30]:
models.values()

dict_values([LinearRegression(), Lasso(), Ridge(), ElasticNet()])

In [31]:
import numpy as np

def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [44]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)
    
    ## make predictions for validation test data
    y_pred = model.predict(X_test)
    
    mae, rmse, r2_square = evaluate_model(y_test, y_pred)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
 
   ## results are for validation set   
    print('Model Training Performance')
    print("RMSE", rmse)
    print("MAE", mae)
    print("r2_score", r2_square*100)
    
    r2_list.append(r2_square)
    
    print('='*35)
    print('\n')
    

LinearRegression
Model Training Performance
RMSE 1013.9047094344004
MAE 674.0255115796833
r2_score 93.68908248567512


Lasso
Model Training Performance
RMSE 1013.8723151049943
MAE 675.2036880701621
r2_score 93.68948574778251


Ridge
Model Training Performance
RMSE 1013.905899776106
MAE 674.0565132295936
r2_score 93.68906766741328


ElasticNet
Model Training Performance
RMSE 1513.9140350229761
MAE 1050.7468664314322
r2_score 85.92978759337907




In [41]:
trained_model_list

[]

In [42]:
model_list

['LinearRegression',
 'LinearRegression',
 'Lasso',
 'Ridge',
 'ElasticNet',
 'LinearRegression',
 'Lasso',
 'Ridge',
 'ElasticNet',
 'LinearRegression',
 'Lasso',
 'Ridge',
 'ElasticNet',
 'LinearRegression',
 'Lasso',
 'Ridge',
 'ElasticNet',
 'LinearRegression',
 'Lasso',
 'Ridge',
 'ElasticNet']

In [43]:
r2_list

[0.9368908248567511,
 0.9368948574778251,
 0.9368906766741328,
 0.8592978759337908,
 0.9368908248567511,
 0.9368948574778251,
 0.9368906766741328,
 0.8592978759337908,
 0.9368908248567511,
 0.9368948574778251,
 0.9368906766741328,
 0.8592978759337908,
 0.9368908248567511,
 0.9368948574778251,
 0.9368906766741328,
 0.8592978759337908,
 0.9368908248567511,
 0.9368948574778251,
 0.9368906766741328,
 0.8592978759337908]